# 😇 Decoder-Only | GPT-Neo 1.3B

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# Load GPT-Neo model and tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

model.safetensors:  44%|####4     | 2.36G/5.31G [00:00<?, ?B/s]